In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

import pandas as pd
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

nlp = spacy.load('en')

dtypes ={
    'id': np.uint32,
    'qid1': np.uint32,
    'qid2': np.uint32,
    'question1': np.str,
    'question2': np.str,
    'is_duplicate': np.uint8
}

df = pd.read_csv('../../dataset/quora_train.csv', dtype=dtypes, usecols=['question1', 'question2', 'is_duplicate'], nrows=1000)

In [2]:
def pad(v):
    zeros = np.zeros([100, 300])
    zeros[:v.shape[0],:v.shape[1]] = v
    return zeros
    
q1s = df['question1'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)
q2s = df['question1'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)

q1m = np.concatenate(q1s.values).flatten().reshape([1000, 100, 300])
q2m = np.concatenate(q2s.values).flatten().reshape([1000, 100, 300])

In [3]:
tf.reset_default_graph()

Q1 = tf.placeholder(tf.float32, shape=[None, 100, 300])
Q2 = tf.placeholder(tf.float32, shape=[None, 100, 300])

cell1 = rnn.BasicRNNCell(128)
cell2 = rnn.BasicRNNCell(128, reuse=True)
state = tf.get_variable(shape=[1000, 128], initializer=tf.zeros_initializer(), name='state')

outputs1, state = tf.nn.dynamic_rnn(cell1, Q1, initial_state=state)
outputs2, state = tf.nn.dynamic_rnn(cell2, Q2, initial_state=state)

output1 = outputs1[:,-1,:]
output2 = outputs2[:,-1,:]

out = tf.concat([output1, output2], axis=1)

In [4]:
W1 = tf.get_variable(shape=[256, 256], initializer=tf.contrib.layers.xavier_initializer(), name='weight1')
b1 = tf.get_variable(shape=[256], initializer=tf.contrib.layers.xavier_initializer(), name='bias1')
y1 = tf.matmul(out, W1) + b1

W2 = tf.get_variable(shape=[256, 1], initializer=tf.contrib.layers.xavier_initializer(), name='weight2')
b2 = tf.get_variable(shape=[1], initializer=tf.contrib.layers.xavier_initializer(), name='bias2')
y2 = tf.matmul(y1, W2) + b2

In [5]:
labels = df['is_duplicate'].values.reshape([-1, 1])
cost = tf.reduce_sum(tf.squared_difference(y2, labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
for i in range(100):
    sess.run(optimizer, feed_dict={Q1: q1m, Q2: q2m})
    print(sess.run(cost, feed_dict={Q1: q1m, Q2: q2m}))

251.035
249.658
248.28
246.903
245.532
244.173
242.835
241.532
240.277
239.091
238.004
237.053
236.29
235.78


KeyboardInterrupt: 